In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [4]:
# Import Production Data (and State Index)
# Production Data Source
# https://www.kaggle.com/datasets/kevinmorgado/us-energy-generation-2001-2022?select=organised_Gen.csv

# Load the State Index CSV file
state_df = pd.read_csv("Resources/states.csv", usecols = ['State', 'Code'])

# Load the Production CSV file and exclude first column
col_list = ['YEAR', 'MONTH', 'STATE', 'TYPE OF PRODUCER', 'ENERGY SOURCE', 'GENERATION (Megawatthours)']
prod_df = pd.read_csv("Resources/organised_Gen.csv", usecols = col_list)

# Assign title case to column names and rename 'State'
col_list_cased = [col.title() for col in col_list]
prod_df.columns = col_list_cased

# Filter for years 2002 - 2021 to match investment data set
prod_df = prod_df.loc[(prod_df['Year'] > 2001) & (prod_df['Year'] < 2022) & ("TYPE OF PRODUCER" == "Total Electric Power Industry")]

# Filter for common energy types across both data sets
prod_source_list = ['Geothermal', 'Hydroelectric Conventional', 'Other Biomass', 'Solar Thermal and Photovoltaic', 'Wind']
prod_df = prod_df.loc[prod_df['Energy Source'].isin(prod_source_list)]

# Rename energy sources to common convention
prod_df = prod_df.replace(['Hydroelectric Conventional', 'Other Biomass', 'Solar Thermal and Photovoltaic'], 
                          ['Hydroelectric', 'Biomass', 'Solar'])

# Display sample data
prod_df.head()

,Year,Month,State,Type Of Producer,Energy Source,Generation (Megawatthours)
20215,2002,1,AK,Total Electric Power Industry,Hydroelectric,118944.0
20217,2002,1,AK,Total Electric Power Industry,Biomass,999.0
20222,2002,1,AK,"Electric Generators, Electric Utilities",Hydroelectric,118944.0
20235,2002,1,AK,"Combined Heat and Power, Industrial Power",Biomass,999.0
20242,2002,1,AL,Total Electric Power Industry,Hydroelectric,947959.0


In [5]:
# Import Investment Data
# Investment Data Source
# https://www.wctsservices.usda.gov/Energy/Downloads

# Load the Investment Excel file, "Detailed" Sheet
inv_df = pd.read_excel("Resources/EnergyInvestments_DataDownloads.xlsx", sheet_name = "Detailed")

# Filter for common energy types across both data sets
inv_source_list = ['Geothermal', 'Hydroelectric', 'Renewable Biomass', 'Solar', 'Wind']
inv_df = inv_df.loc[inv_df['Energy Type'].isin(inv_source_list)]

# Rename energy sources to common convention
inv_df = inv_df.replace('Renewable Biomass', 'Biomass')

# Join with State Index data to get State Abbreviation column
inv_df = pd.merge(inv_df, state_df, on = "State")

# Rename columns
inv_df.rename(columns={"State": "State Name", "Code": "State", "Energy Type": "Energy Source", "Program_Name": "Program Name"}, inplace = True)

# Display sample data
inv_df.head()

,State Name,County,Congressional District,Zip Code,Energy Source,Agency,Year,Program Name,Total Number of Investments,Total Amount of Assistance,Description,State
0,Alabama,Baldwin,AL-01,36507,Biomass,FSA,2010,BCAP Matching Payments (NOFA),2,40736.0,NaN,AL
1,Alabama,Baldwin,AL-01,36551,Biomass,FSA,2010,BCAP Matching Payments (NOFA),1,16509.0,NaN,AL
2,Alabama,Baldwin,AL-01,36567,Biomass,FSA,2010,BCAP Matching Payments (NOFA),1,639.0,NaN,AL
3,Alabama,Baldwin,AL-01,36580,Biomass,FSA,2010,BCAP Matching Payments (NOFA),1,16334.0,NaN,AL
4,Alabama,Baldwin,AL-01,36580,Biomass,RBS,2021,Rural Energy for America Program (REAP),1,7000000.0,Landfill Gas,AL


In [ ]:
#### Exploratory Data Analysis

# Group by year, state, congressional district, zip, energy source, agency
# ANOVA by state